# Exploring EV Fuel Economy

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import requests

In [2]:
df = pd.read_csv('assets/vehicles.csv')

df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [3]:
df.columns

Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'guzzler',
       'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2', 'rangeA',
       'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c240bDscr',
       'createdOn', 'modifiedOn

In [4]:
#df = df[['rangeHwyA', 'rangeCityA', 'make', 'model', 'year', 'mpgData', 'charge240', 'fuelCost08', 'fuelType1', ]]


elec_df = df[df['fuelType1'] == 'Electricity']

elec_df = elec_df[['range', 'rangeHwy', 'rangeCity', 'highway08', 'make', 'model', 'year',
                   'charge120', 'charge240', 'fuelCost08', 'fuelType1']]
# range = rounded, weighted avg of City (55%) and Hwy (45%) driving 
# highway08 = highway MPGe (equivalent for EVs to mpg)
# charge120/charge240 = time to charge an EV in hours at 120V/240V
# fuelCost08 = annual fuel cost based on 15k miles, 55% city driving, and price of fuel

#elec_df = elec_df[elec_df['year'] >= 2012] # only after 2012 did they have data on range

elec_df = elec_df.sort_values('year')


elec_df.head()

,range,rangeHwy,rangeCity,highway08,make,model,year,charge120,charge240,fuelCost08,fuelType1
23028,33,0.0,0.0,58,Chevrolet,S10 Electric,1998,0.0,0.0,1200,Electricity
23029,72,0.0,0.0,28,Chevrolet,S10 Electric,1998,0.0,0.0,2350,Electricity
23027,81,0.0,0.0,46,Honda,EV Plus,1998,0.0,0.0,1400,Electricity
23026,70,0.0,0.0,33,Plymouth,Voyager/Grand Voyager 2WD,1999,0.0,0.0,1850,Electricity
23025,70,0.0,0.0,33,Dodge,Caravan/Grand Caravan 2WD,1999,0.0,0.0,1850,Electricity


NOTE: Standard homes have 240V, but most wall outlets are only 120, so it is possible to have a level 2 charge at home but need a special plug-in station installed. 

## EV Range vs Annual Fuel Cost*

*Fuel Cost based on 15,000 miles travelled, 55% city driving, and prices for type of fuel (electric)

In [5]:
#gas_df = df[df['fuelType1'] == 'Gasoline']
#df['fuelType1'].unique()
gas_df = df[df['fuelType1'] != 'Electricity']
gas_df = gas_df[gas_df['year'] >= 2018]


gas_df['fuelCost08'].mean()

1820.2636419374617

In [6]:
scatter = alt.Chart(elec_df).mark_point(filled=True).encode(
    x = alt.X('fuelCost08', axis=alt.Axis(format='$.0f', title='Annual Fuel Cost')),
    y = alt.Y('range', axis=alt.Axis(title='Range (miles)')),
    color = alt.Color('year:O', scale=alt.Scale(scheme='yellowgreen', domain=[i for i in range(1998,2022)])),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).properties(title ='EV Range vs. Annual Fuel Cost').interactive()


gas_mean = alt.Chart(gas_df).mark_rule(color='red', strokeDash=[10,5]).encode(
    x = 'mean(fuelCost08):Q'
)

range_v_fuelcost = scatter + gas_mean

range_v_fuelcost.save("range_v_fuelcost.html") 
from IPython.display import IFrame
IFrame(src='range_v_fuelcost.html', width=850, height=450)

## Annual Fuel Cost vs. Model Year

In [7]:
alt.Chart(elec_df).mark_point(filled=True).encode(
    x = alt.X('year', axis=alt.Axis(format='') , scale = alt.Scale(domain=(1997,2022))),
    y = alt.Y('fuelCost08'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).interactive()

alt.Chart(...)

## EV Range vs. Model Year

In [8]:
alt.Chart(elec_df).mark_point(filled=True).encode(
    x = alt.X('year', axis=alt.Axis(format='') , scale = alt.Scale(domain=(1997,2022))),
    y = alt.Y('range'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).interactive()

alt.Chart(...)

The below chart is same as above, but also encodes make using color and shape

In [9]:
viable_df = elec_df[elec_df['range'] >= 100]
viable_df = viable_df[viable_df['year'] >= 2008]

range_v_modelyear = alt.Chart(viable_df).mark_point(filled=True, size=50, opacity=0.6).encode(
    x = alt.X('year', axis=alt.Axis(format='', title='Model Year') , scale = alt.Scale(domain=(2006,2022))),
    y = alt.Y('range', axis=alt.Axis(title='Range (miles)')),
    color = alt.Color('make:N', sort='ascending', scale=alt.Scale(scheme='tableau20')),
    shape = alt.Shape('make:N', sort='ascending'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).properties(title ='EV Range vs. Model Year').interactive()

#Display the chart and save
range_v_modelyear.save("range_v_modelyear.html") 
from IPython.display import IFrame
IFrame(src='range_v_modelyear.html', width=850, height=450)

Same as above, but limited to only EVs with a range > 150 miles.

In [10]:
usable_df = elec_df[elec_df['range'] > 150]

alt.Chart(usable_df).mark_point(filled=True).encode(
    x = alt.X('year', axis=alt.Axis(format='') , scale = alt.Scale(domain=(1997,2022))),
    y = alt.Y('range'),
    color = alt.Color('make:N'),
    shape = alt.Shape('make:N'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).properties(title ='EV Range vs. Model Year').interactive()

alt.Chart(...)

Same chart, but only Tesla included

In [11]:
tesla_df = elec_df[elec_df['make'] == 'Tesla']

alt.Chart(tesla_df).mark_point(filled=True).encode(
    x = alt.X('year', axis=alt.Axis(format='') , scale = alt.Scale(domain=(1997,2022))),
    y = alt.Y('range'),
    color = alt.Color('make:N'),
    shape = alt.Shape('make:N'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).interactive()

alt.Chart(...)

## Level 2 Charging Time vs. Range

In [12]:
charge_v_range = alt.Chart(viable_df).mark_point(filled=True, size=50, opacity=0.6).encode(
    x = alt.X('range', axis=alt.Axis(title='Range (miles)')),
    y = alt.Y('charge240', axis=alt.Axis(title='Level 2 Charging Time (hours)')),
    color = alt.Color('make:N', sort='ascending', scale=alt.Scale(scheme='tableau20')),
    shape = alt.Shape('make:N', sort='ascending'),
    tooltip = ['make', 'model', 'year', 'fuelCost08', 'range', 'charge240']
).properties(title ='Level 2 Charging Time vs. EV Range').interactive()

#Display the chart and save
charge_v_range.save("chargetime_v_range.html") 
from IPython.display import IFrame
IFrame(src='chargetime_v_range.html', width=850, height=450)